# Práctica 2: Aprendizaje y selección de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

### Realizado por:

* Antonio Beltrán Navarro
* Ramón Jesús Martínez Sánchez

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo

En esta práctica estudiaremos los modelos más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los clasificadores resultantes. Además, veremos métodos de selección de modelos orientados a obtener una configuración óptima de hiperparámetros.

# Breast Cancer Wisconsin

# 1. Preliminares

Lo primero de todo es cargar las librerías para que estén disponibles posteriormente:

In [ ]:
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.impute import SimpleImputer
import numpy as np

# Local application
import utilidad_grupor as utils

Fijaremos tambíen una semilla aleatoria para que los experimentos sean reproducibles:

In [ ]:
seed = 27912

# 2. Carga de datos

Comenzamos cargando el conjunto de datos `wisconsin`:

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

data = utils.load_data(filepath, index, target)

Este conjunto de datos está compuesto por 30 variables predictoras que en realidad se agrupan en 10 ya que estas se ven divididas entre `_mean, _se y _worst`, una variable `Unnamed` que será tratada posteriormente y la variable clase `diagnosis` que nos indicará si el tumor es **maligno** (m) o **benigno** (b).

Las variables predictoras, por tanto, son:
* `radius, perimeter, area, compactness`: relacionadas con el tamaño del tumor
* `symmetry`: se refiere a la simetría en la forma del tumor
* `smoothness`: variación en la longitud de los tamaños
* `concavity, concave_points`: relacionados con la concavidad del tumor
* `texture`: referente a la textura del tumor (en escala de grises)
* `fractal_dimension`

Una vez hemos cargado el conjunto de datos, mostraremos 5 registros aleatorios mediante la función `sample` para comprobar que el proceso ha sido realizado correctamente

In [ ]:
data.sample(5, random_state=seed)

Como podemos observar, tenemos una columna, la última, cuyo nombre es `Unnamed` y todo el contenido de sus filas `NaN`. Esto es debido a que en la declaración de las columnas en el archivo `csv` hay una `,` sobrante al final de la línea, lo que hace que se cree una columna sin nombres y con valores inexistentes.

Esto significa que antes de continuar trabajando con nuestro conjunto de datos, debemos borrar esa columna.

In [ ]:
del data['Unnamed: 32']

Comprobamos que se ha borrado correctamente haciendo uso del método `sample` de nuevo:

In [ ]:
data.sample(5, random_state=seed)

In [ ]:
data.diagnosis.unique()

Con el propósito de usar métricas como el `recall` en el apartado de selección de modelos, es necesario convertir nuestra variable clase que resulta ser categórica a una con valores numéricos.

Para ello elegiremos que la clase **M** (malign) pasará a tomar un valor de 1, y la clase **B** (benign) pasará a tomar un valor de 0.

Mas adelante veremos que dicho cambio se ha realizado correctamente.

In [ ]:
data['diagnosis']=data['diagnosis'].map({'M':1,'B':0})

A continuación, separaremos en dos subconjuntos nuestro conjunto de datos inicial, uno con las variables predictoras (`X`) y otro con la variable objetivo (`y`). 

In [ ]:
(X, y) = utils.divide_dataset(data, target="diagnosis")

Comprobaremos que se hayan separado correctamente:

Empezamos mostrando las variables predictoras:

In [ ]:
X.sample(5, random_state=seed)

Y a continuación la variable clase:

In [ ]:
y.sample(5, random_state=seed)

Antes de comenzar el análisis exploratorio de los datos, dividiremos nuestro conjunto de datos en otros dos subconjuntos, uno de entrenamiento y otro de prueba, con los siguientes porcentajes:

* Conjunto de entrenamiento: **70%**
* Conjunto de prueba: **30%**

Mediante este proceso, nos aseguraremos de que los resultados posteriores del proceso de validación han sido obtenidos de una manera correcta.

In [ ]:
train_size = 0.7
(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=y,
                                                      random_state=seed,
                                                      train_size=train_size)

Seguidamente, lo que haremos será unir los conjuntos `X_train` e `y_train` para obtener el conjunto de datos de entrenamiento.
Haremos los mismo para `X_test` e `y_test`, juntando así el conjunto de datos de test.

In [ ]:
train = utils.join_dataset(X_train, y_train)
test = utils.join_dataset(X_test, y_test)

Nuevamente, vamos a asegurarnos de que el conjunto de datos se ha dividido correctamente. Para ello visualizaremos el conjunto de datos de entrenamiento, observando que la variable clase también aparece al final del conjunto:

In [ ]:
train.sample(5, random_state=seed)

Una vez tenemos bien definidos nuestros conjuntos de entrenamiento y prueba, podemos aplicar el **preprocesamiento de datos** definido en la práctica anterior.

## 2.1. Preprocesamiento de datos

### 2.1.1. Eliminación de variables

Para realizar este proceso, haremos uso de un **Pipeline**. Este Pipeline será el encargado de aplicar las transformaciones que hemos decidido a nuestro conjunto de datos.

Eliminaremos las siguientes columnas (variables):
* Aquellas relacionadas con los `concave points`: `concavity_mean, compactness_mean, concavity_se, compactness_se, concavity_worst y compactness_worst`.
* Aquellas relacionadas con `radius`, es decir: `area_mean, perimeter_mean, area_se, perimeter_se, area_worst y perimeter_worst`.

Porque son variables que dependen directamente de las 2 que vamos a dejar en nuestro conjunto de datos: `radius y concave points`.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

preproc = ColumnTransformer([("", "drop", ["concavity_mean", "compactness_mean", "concavity_se", "compactness_se", "concavity_worst", "compactness_worst", "area_mean", "perimeter_mean", "area_se", "perimeter_se", "area_worst", "perimeter_worst"])], remainder="passthrough")

### 2.1.2. Normalización

Será necesario **normalizar** el conjunto de datos a la hora de definir los modelos a emplear, de forma que todas las variables predictoras tengan el mismo peso.

Para ello definiremos una variable `normalizacion` que haga uso de la función `Normalizer()`, la cual normaliza individualmente los datos a una norma unitaria, justo lo que buscamos.

In [ ]:
normalizacion = Normalizer()

Una vez podamos añadir al **Pipeline** el preprocesamiento de datos obtenido, podemos pasar a tratar los modelos de clasificación para nuestro conjunto de datos.

# 3. Modelos de clasificación supervisada

Es el momento de definir los diferentes modelos que utilizaremos en esta práctica. Cada uno de estos modelos contará con el preprocesamiento de datos definido previamente haciendo uso del **Pipeline**.

Comentaremos los parámetros de cada modelo más importantes para nuestro problema y que utilizaremos posteriormente mediante un proceso de validación cruzada.

## 3.1. Vecinos más cercanos

Este algoritmo es considerado perozoso puesto que computa los parámetros necesarios para la clasificación durante inferencia. La clasificación de este modelo consistirá en asignar a la instancia de entrada la clase mayoritaria de los vecinos más cercanos.

Encontramos 2 principales parámetros a la hora de definir nuestro modelo de vecinos mas cercanos, el número de vecinos `n_neighbours` que se dejará por defecto en un valor moderado como puede ser 5, y los pesos `weights`, donde consideraremos que todos los vecinos tienen la misma importancia, y por tanto, el mismo peso. Dejaremos los valores de esta variable por defecto.

Por último, al crear el **Pipeline** de este algoritmo, deberemos pasarle las variables normalizadas como hemos definido previamente en el preprocesamiento de datos, de esta manera escalaremos las variables predictoras en proporciones similares que serán tratadas por los pesos.

In [ ]:
n_neighbors = 5
weights = 'distance'

k_neighbors_model = make_pipeline(
        preproc,
        normalizacion,
        KNeighborsClassifier(n_neighbors, weights=weights)
)

## 3.2. Árboles de decisión

Los algoritmos basados en inducción de árboles de decisión representan los modelos mediante un conjunto de reglas.

Los hiperparámetros que a nuestro parecer resultan más interesantes para este modelo serán `max_depth`, que tomará un valor de 3 y se referirá a la profundidad del arbol (evitaremos sobreajuste con este valor), y `min_samples_split`, que tomará un valor de 20, lo que significa que las hojas deberán tener un mínimo de 20 instancias para evitar, de nuevo, un posible sobreajuste.

Es necesario comentar que posteriormente, en el apartado de evaluación de modelos, estudiaremos de mejor forma estos hiperparámetros.

Vamos a configurar nuestro árbol de decisión con lo mencionado anteriormente:

In [ ]:
decision_tree_model = make_pipeline(
        preproc,
        DecisionTreeClassifier(random_state=seed, 
                               max_depth=3,
                               criterion='entropy',
                               ccp_alpha=0.1,
                               min_samples_leaf=25)
) 

## 3.4. Adaptative Boosting (*AdaBoost*)

Este algoritmo propone un entrenamiento de una serie de clasificadores de manera iterativa, de modo que cada nuevo clasificador se enfoque en los datos que fueron erróneamente clasificados por su predecesor, de esta forma el algoritmo se adapta y logra obtener mejores resultados.

Los hiperparámetros más relevantes a tener en cuenta serán `base_estimator`, que nos indica el estimador que utilizará el ensemble (en este caso un Árbol de decisión), el número de estimadores `n_estimators`, o la tasa de aprendizaje que nos ayudará a controlar la contribución de cada estimador `learning_rate`.

Es necesario comentar que posteriormente, en el apartado de evaluación de modelos, estudiaremos de mejor forma estos hiperparámetros.


Vamos a configurar un modelo AdaBoost sencillo que utilice los hiperparámetros por defecto:

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=seed)

adaboost_model = make_pipeline(
        preproc,
        AdaBoostClassifier(random_state=seed, base_estimator=DecisionTreeClassifier(max_depth=1))
)

## 3.5. Bootstrap Aggregating (*Bagging*)

Bagging es un ensemble cuya estrategia es utilizar una función de aprendizaje y obtener modelos diversos entre sí para reducir el error obtenido mediante varianza. Mediante un voto de estos modelos se obtendrá el ensemble.

Los hiperparámetros más relevantes a tener en cuenta serán `base_estimator`, que nos indica el estimador que utilizará el ensemble (en este caso un Árbol de decisión) o el número de estimadores `n_estimators`. Para un muestreo con reemplazo, emplearemos el parámetro `bootstrap_features=True`.defecto, `random_state=None`).

Es necesario comentar que posteriormente, en el apartado de evaluación de modelos, estudiaremos de mejor forma estos hiperparámetros.


Vamos a configurar un ensemble tipo Bagging utilizando los hiperparámetros por defecto:

In [ ]:
bagging_model = make_pipeline(
        preproc,
        BaggingClassifier(random_state=seed, base_estimator=DecisionTreeClassifier(max_depth=None))
)

## 3.6. Random Forests

Un Random Forest es un conjunto (ensemble) de árboles de decisión combinados con bagging. Al usar bagging, lo que en realidad está pasando, es que distintos árboles ven distintas porciones de los datos. Ningún árbol ve todos los datos de entrenamiento. Esto hace que cada árbol se entrene con distintas muestras de datos para un mismo problema. De esta forma, al combinar sus resultados, unos errores se compensan con otros y tenemos una predicción que generaliza mejor.

Vamos a configurar un estimador tipo Random Forests usando los hiperparámetros por defecto:

In [ ]:
random_forest_model = make_pipeline(
        preproc,
        RandomForestClassifier(random_state=seed)
)

## 3.7. Gradient Tree Boosting (*Gradient Boosting*)

Gradient Boosting es una generalización de los algoritmos de Boosting con la capacidad de optimizar cualquier tipo de función pérdida, generando un conjunto de estimadores de forma secuencial. No todos estos estimadores tendrán la misma importancia, puesto que en cada iteración solo se tomarán en cuenta a los modelos anteriores.



Vamos a configurar este estimador usando los hiperparámetros por defecto:

In [ ]:
gradient_boosting_model = make_pipeline(
        preproc,
        GradientBoostingClassifier(random_state=seed)
)

## 3.8. Histogram-Based Gradient Boosting (*Histogram Gradient Boosting*)

El algoritmo Histogram Gradient Boosting es una optimización de *Gradient Boosting* que discretiza el conjunto de datos de entrada con el fin de poder trabajar con un mayor número de instancias en un tiempo razonable.

Al contrario que en otros modelos, Histogram Gradient Boosting realiza su propia discretización, por lo que no tendremos que proporcionársela mediante el **Pipeline**.

Vamos a usar los hiperparámetros por defecto para configurar este estimador:

In [ ]:
hist_gradient_boosting_model = make_pipeline(
        preproc,
        HistGradientBoostingClassifier(random_state=seed)
)

# 4. Evaluación de modelos

A la hora de evaluar los distintos modelos, utilizaremos una técnica conocida como **validación cruzada**. En esta, se separa el conjunto de datos en `k` particiones y se repite `k` veces el proceso de aprendizaje y validación, pero utilizando cada vez una combinación única de `k-1` muestras para entrenar y la restante para validar. De este modo, obtendremos unos valores fiables de sesgo empleando solamente el conjunto de entrenamiento.

En este caso vamos a evaluar nuestros clasificadores y el tipo de validación cruzada a utilizar usando una `5*10-cv` estratificada:

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=seed)

## 4.1. Árbol de decisión

In [ ]:
model = decision_tree_model

utils.evaluate_estimator(model, X_train, y_train, cv)

## 4.2. Vecinos más cercanos

In [ ]:
model = k_neighbors_model

utils.evaluate_estimator(model, X_train, y_train, cv)

## 4.3. Adaptative Boosting (*AdaBoost*)

In [ ]:
model = adaboost_model

utils.evaluate_estimator(model, X_train, y_train, cv)

## 4.4. Bootstrap Aggregating (*Bagging*)

In [ ]:
model = bagging_model

utils.evaluate_estimator(model, X_train, y_train, cv)

## 4.5. Random Forests

In [ ]:
model = random_forest_model

utils.evaluate_estimator(model, X_train, y_train, cv)

## 4.6. Gradient Tree Boosting (Gradient Boosting)

In [ ]:
model = gradient_boosting_model

utils.evaluate_estimator(model, X_train, y_train, cv)

## 4.7. Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
model = hist_gradient_boosting_model

utils.evaluate_estimator(model, X_train, y_train, cv)

# 5. Selección de modelos

Ahora que somos capaces de evaluar correctamente los clasificadores, es importante decidir una estrategia que nos permita encontrar una configuración óptima de los hiperparámetros. Para ello haremos uso de una búsqueda **Grid** para explorar las posibles combinaciones de hiperparámetros y obtener el mejor modelo.

Definimos las 2 métricas que usaremos para seleccionar nuestros modelos, como son el `accuracy` y el `recall`. A la hora de elegir los mejores modelos, seleccionaremos aquellos que logren un mejor resultado en recall, el ratio de verdaderos positivos y falsos negativos obtenido mediante $\frac {tp}{(tp+fn)}$

In [ ]:
scoring = ["accuracy","recall"]

Además de usar estas dos métricas, emplearemos el método refit que nos servirá para elegir cuál es el mejor modelo una vez acabada la búsqueda **Grid** y volver a entrenar con él tratando de mejorar, en este caso, su **recall**.

## 5.1. Árbol de decisión

En esta selección de modelos trataremos de optimizar los parámetros que consideramos son más relevantes en árboles de decisión.

Los hiperparámetros elegidos para ser optimizados han sido:
* `max_depth`: Profundidades del árbol que se estudiarán para evitar un posible sobreajuste y garantizar un buen resultado.
* `ccp_alpha`: Parámetros de complejidad usado para una posibilidad de poda posterior con el mínimo coste computacional
* `criterion`: Función para medir la calidad de las particiones del árbol. Usaremos `entropy` para la ganancia de información y el índice `gini`.
* `min_samples_leaf`: Mínimo número de instancias para poder tener en cuenta un nodo hoja, lo cual afecta al suavizado del modelo que sea elegido. 

In [ ]:
estimator = decision_tree_model

max_depth = [3, 4, 5, 6, 7]
ccp_alpha = [0, 0.05, 0.1, 0.2]
criterion = ['entropy', 'gini']
min_samples_leaf = [5, 10, 15, 20, 25]

decision_tree_clf = utils.optimize_params(estimator,
                                        X_train, y_train, cv,
                                        scoring=scoring, refit="recall",
                                        decisiontreeclassifier__max_depth=max_depth,
                                        decisiontreeclassifier__criterion=criterion,
                                        decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                        decisiontreeclassifier__min_samples_leaf=min_samples_leaf)

Una vez se ha realizado la búsqueda **Grid**, vemos que el modelo que nos proporciona mejores resultados cuenta con un `ccp_alpha` de 0, lo que nos indica que una posible post poda no ayudaría a mejorar el modelo. Además vemos que el criterio elegido es el de entropía que trata de maximizar la ganancia de información, una profundidad del árbol de 3, y un número mínimo de ejemplos por nodo hoja de 10.

## 5.2. Vecinos más cercanos

En esta selección de modelos trataremos de optimizar los parámetros que consideramos son más relevantes en el algoritmo de vecinos más cercanos.

Los hiperparámetros elegidos para ser optimizados han sido:
* `weights`: Función de peso de los vecinos, que pueden ser uniformes o por distancias.
* `n_neighbors`: Número de vecinos más cercanos para elegir la clase del que se estudia. Un gran número de vecinos podría llevar al modelo a parecerse a un `ZeroR`, y un pequeño número de vecinos podría causarnos un gran sobreajuste. Es por ello que este hiperparámetro es el más importante a la hora de seleccionar nuestro modelo de vecinos más cercanos.

In [ ]:
estimator = k_neighbors_model

weights = ["uniform", "distance"]
n_neighbors = [3, 5, 7, 9, 11, 13, 15]


k_neighbors_clf = utils.optimize_params(estimator,
                                        X_train, y_train, cv, scoring=scoring, refit="recall",
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors)

La estrategia seleccionada utiliza una distribución de pesos uniforme, y un número de vecinos muy pequeño: 3. Es interesante que éste sea el número de vecinos, puesto que es el menor de los proporcionados y hemos comentado que un pequeño número de vecinos podría causarnos sobreajuste.

## 5.3. Adaptative Boosting (AdaBoost)

En esta selección de modelos trataremos de optimizar los parámetros que consideramos son más relevantes en AdaBoost.

Los hiperparámetros elegidos para ser optimizados han sido:
* `criterion`: Función para medir la calidad de las particiones del árbol. Usaremos `entropy` para la ganancia de información y el índice `gini`.
* `learning_rate`: Nos indica la contribución de los estimadores basada en el valor de esta variable, por defecto a 1.
* `min_samples_split`: Mínimo número de instancias requeridas para dividir un nodo interno, este hiperparámetro será usado por los árboles que conforman nuestro modelo de AdaBoost.

In [ ]:
estimator = adaboost_model

criterion = ["gini", "entropy"]
learning_rate = [0.25, 0.5, 0.75, 1]
min_samples_split = [2, 5, 10, 20, 30]

adaboost_clf = utils.optimize_params(estimator,
                                        X_train, y_train, cv, scoring=scoring, refit="recall",
                                        adaboostclassifier__base_estimator__min_samples_split=min_samples_split,
                                        adaboostclassifier__base_estimator__criterion=criterion,
                                        adaboostclassifier__learning_rate=learning_rate)

En cuanto al criterio seleccionado, vemos que en este caso el índice gini ha conseguido darnos mejores resultados en nuestro modelo junto a una tasa de aprendizaje (learning rate) que mantiene su valor por defecto de 1.

Conociendo el algoritmo AdaBoost sabemos que éste trabaja con clasificadores 1R, los cuales reciben una inmensa cantidad de instancias, no obstante es interesante remarcar que el hiperparámetro `min_samples_split` para los nodos internos ha resultado no ser relevante para el problema.

## 5.4. Bootstrap Aggregating (Bagging)

En esta selección de modelos trataremos de optimizar los parámetros que consideramos son más relevantes en Bagging.

Los hiperparámetros elegidos para ser optimizados han sido:
* `max_samples`: Número de instancias del conjunto de entrenamiento necesarias para entrenar cada estimador.
* `max_features`: Número de variables predictoras de nuestro conjunto de entrenamiento necesarias para entrenar cada estimador.

Ambas variables toman por defecto el valor 1.

In [ ]:
estimator = bagging_model

max_samples = [0.25, 0.5, 0.75, 1]
max_features = [0.25, 0.5, 0.75, 1]

bagging_clf = utils.optimize_params(estimator,
                                     X_train, y_train, cv, scoring=scoring, refit="recall",
                                     baggingclassifier__max_samples=max_samples,
                                     baggingclassifier__max_features=max_features)

De esta manera, podremos apreciar los valores de los hiperparámetros que optimizan las métricas del modelo seleccionado, en este caso `max_features = 0.75` y `max_samples = 0.5`

## 5.5. Random Forests

Como hemos comentado previamente, este algoritmo emplea árboles distintos con mucho sobreajuste, por lo que no resultaría de importancia controlar los hiperparámetros de éstos árboles como hicimos en AdaBoost. Nos limitaremos a controlar los siguientes hiperparámetros referentes a los Random Forest:
* `max_samples`: Número de instancias del conjunto de entrenamiento necesarias para entrenar cada estimador (árbol).
* `max_features`: Número máximo de variables para considerar al buscar la mejor división al entrenar los árboles. Al utilizar `sqrt` estamos indicando que `max_features=sqrt(n_features)`. Lo mismo para `log2`, tal que `max_features=log2(n_features)`
* `criterion`: Función para medir la calidad de las particiones del árbol. Usaremos `entropy` para la ganancia de información y el índice `gini`.

In [ ]:
estimator = random_forest_model

max_samples = [0.25, 0.5, 0.75]
max_features = ['sqrt', 'log2']
criterion = ["gini", "entropy"]

random_forest_clf = utils.optimize_params(estimator,
                                               X_train, y_train, cv, scoring=scoring, refit="recall",
                                               randomforestclassifier__max_samples=max_samples,
                                               randomforestclassifier__criterion=criterion,
                                               randomforestclassifier__max_features=max_features)

El modelo seleccionado utilizará un 50% de la instancias para entrenar los árboles, junto con un número máximo de variables guiado por `sqrt`, y haciendo uso de un criterio basado en el índice `gini`, garantizando así un modelo con un recall y una accuracy adecuados para nuestro problema.

## 5.6. Gradient Tree Boosting (Gradient Boosting)

En esta selección de modelos trataremos de optimizar los parámetros que consideramos son más relevantes en Gradient Boosting.

Los hiperparámetros elegidos para ser optimizados han sido:
* `learning_rate`: Tasa de aprendizaje usada como factor multiplicativo para los nodos hoja. Toma un valor por defecto de 0.1.
* `max_depth`: Profundidad máxima de cada árbol (desde la raíz al nodo más profundo). Este parámetro no se comprueba por defecto, pero puede resultar de interés estudiarlo.


In [ ]:
estimator = gradient_boosting_model

learning_rate = [0.025, 0.05, 0.1]
max_depth = [1, 3, 5, 7]


gradient_boosting_clf = utils.optimize_params(estimator,
                                               X_train, y_train, cv, scoring=scoring, refit="recall",
                                               gradientboostingclassifier__learning_rate = learning_rate,
                                               gradientboostingclassifier__max_depth=max_depth)

La tasa de aprendizaje (learning rate), se ha dejado a 0.025, mientras que la profundidad del árbol parece no ser demasiado importante. Puesto que toma un valor de 3, podemos extraer que árboles más pequeños trabajan mejor con nuestro algoritmos que árboles que puedan tener una mayor profundidad y por tanto un mayor sobreajuste con nuestro conjunto de entrenamiento.

## 5.7. Histogram-Based Gradient Boosting (Histogram Gradient Boosting)



En esta selección de modelos trataremos de optimizar los parámetros que consideramos son más relevantes en Histogram Gradient Boosting.

Los hiperparámetros elegidos para ser optimizados han sido:
* `min_samples_leaf`: Mínimo número de instancias para poder tener en cuenta un nodo hoja, lo cual afecta al suavizado del modelo que sea elegido. 
* `learning_rate`: Tasa de aprendizaje usada como factor multiplicativo para los nodos hoja. Toma un valor por defecto de 0.1.


In [ ]:
estimator = hist_gradient_boosting_model

min_samples_leaf = [10, 20, 30, 40]
learning_rate = [0.05, 0.1, 0.15]

hist_gradient_boosting_clf = utils.optimize_params(estimator,
                                                  X_train, y_train, cv, scoring=scoring, refit="recall",
                                                  histgradientboostingclassifier__learning_rate=learning_rate,
                                                  histgradientboostingclassifier__min_samples_leaf=min_samples_leaf)

Las conclusiones que podemos obtener son que la tasa de aprendizaje es un factor relevante a la hora de seleccionar un modelo, y que un árbol con un número de instancias en cada nodo hoja nos proporciona mejores resultados.

# 5. Construcción y validación del modelo final

Una vez que en el apartado anterior hemos obtenido mediante la búsqueda **grid** todos los modelos, podemos emplearlos contra el conjunto de test que reservamos al principio del documento para evaluar cuál nos proporciona mejores resultados en las métricas elegidas.

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
utils.evaluate_estimators(estimators, X_test, y_test)

# Conclusiones

De acuerdo con estos resultados, podemos concluir que los **árboles de decisión** y los ensembles como **Random Forest, Gradient Boosting** e **Histogram Gradient Boosting** son los clasificadores que nos ofrecen un mejor rendimiento en términos de **recall** en nuestro conjunto de datos `wisconsin`.

No obstante, pese a centrarnos en el **accuracy** y sobre todo en el **recall**, es importante destacar y tener en cuenta otros factores como el tiempo de aprendizaje y de inferencia.

Si bien los ensembles mencionados anteriormente mejoran ligeramente el recall y la precisión cuando los enfrentamos contra el conjunto de test, los **árboles de decisión** tienen un tiempo de aprendizaje e inferencia mucho menor que el resto de modelos de ensembles estudiados.

Por ello, antes de decantarnos por un modelo u otro para nuestro problema, debemos tener en cuenta si el aumento de coste computacional necesario para emplear un modelo más complejo compensa la ligera mejora que este supone frente a modelos más sencillos.

En el caso del problema que estudiamos, al ser una mejoría tan liviana, no nos resulta necesario aumentar el tiempo de aprendizaje e inferencia de ensembles como Random Forest, Gradient Boosting e Histogram Gradient Boosting, por lo que el modelo seleccionado finalmente han sido los **Árboles de Decisión**.

---

# Pima Diabetes

# 1. Preliminares

Primero cargaremos toda la funcionalidad que necesitaremos:
* Los distintos algoritmos que aprenderemos (`AdaBoostClassifier`, 
`BaggingClassifier`, 
`GradientBoostingClassifier`, 
`HistGradientBoostingClassifier`, 
`RandomForestClassifier`, 
`KNeighborsClassifier` y 
`DecisionTreeClassifier`)

* Funcionalidad relacionada con la selección y evaluación de modelos (`RepeatedStratifiedKFold` y 
`train_test_split`)

Lo siguiente que debemos hacer es fijar una semilla con el objetivo de que todo lo que hagamos sea reproducible.

In [ ]:
random_state = 27912

Ahora cargaremos el conjunto de datos, que es el conjunto [Pima Indian Diabetes](https://www.kaggle.com/uciml/pima-indians-diabetes-database). Con el objetivo de evaluar los modelos que obtendremos finalmente, reservaremos un $30\%$ de los ejemplos para realizar una validación *holdout*.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

target = "Outcome"

data = utils.load_data(filepath, None, target)

train, test = train_test_split(data, train_size=0.7, stratify=data[target], random_state=random_state)

train_X, train_y = utils.divide_dataset(train, target)
test_X, test_y = utils.divide_dataset(test, target)

train.sample(5)


# 2. Preprocesamiento de datos

Como averiguamos en la práctica anterior, existen multitud de valores perdidos (codificados como $0$), que debemos tratar. Para ello, imputaremos por la media en las variables `Glucose`, `BloodPressure` y `BMI`; y eliminaremos las variables `Insulin` y `SkinThickness` que tienen un porcentaje demasiado alto de valores perdidos como para ser significativas.

Ya que dicho proceso no depende del algoritmo que vayamos a usar, lo aplicaremos directamente sobre el conjunto de entrenamiento a priori una sola vez, con el objetivo de ahorrar tiempo entrenando.

In [ ]:
mean_imp = SimpleImputer(missing_values=0)
remove_columns = ColumnTransformer([('a', 'drop', ['Insulin', 'SkinThickness']),
                                    ('b', mean_imp, ['Glucose', 'BloodPressure', 'BMI'])],
                                    remainder='passthrough')
remove_columns.fit_transform(train_X)
remove_columns.fit_transform(test_X)

train_X.sample(5)


# 3. Selección de modelos

En esta práctica, aplicaremos distintos algoritmos de aprendizaje de clasificadores con el objetivo de encontrar el que mejor funciona para el problema en cuestión (lograr predecir que un paciente tiene diabetes). Estos son:

* Vecinos más cercanos

* Árbol de decisión

* AdaBoost

* Bagging

* Random Forests

* Gradient Boosting

* Histogram Gradient Boosting

Por cada algoritmo, buscaremos los hiperparámetros que mejor funcionen mediante el algoritmo de búsqueda en Grid. La evaluación de los modelos obtenidos se hará mediante una $10 \times 5$ validación cruzada, usando como métrica de rendimiento el *recall*, al tratarse de un problema desbalanceado.

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=random_state)

## 3.1. Vecinos más cercanos

Para este algoritmo variaremos dos de los hiperparámetros principales:

* `n_neighbors`: la cantidad de ejemplos que se consideran para clasificar. Lo variaremos de $1$ a $15$

* `weights`: el peso de cada ejemplo. Probaremos con pesos uniformes o proporcionales a la inversa de la distancia.

In [ ]:
k_neighbors_classifier = KNeighborsClassifier()
n_neighbors = np.arange(1, 16)
weights = ["uniform", "distance"]

k_neighbors_clf = utils.optimize_params(k_neighbors_classifier, train_X, train_y, cv=cv, scoring="recall", weights=weights, n_neighbors=n_neighbors)

Como podemos observar, se obtiene un mejor resultado con un valor de `n_neighbors` alto (11) y con una distribución de `weights` uniforme.

## 3.2. Árbol de decisión

Para este algoritmo probaremos distintas opciones para los hiperparámetros:

* `criterion`: el criterio para hacer las particiones (`gini` o `entropy`).

* `max_depth`: la profundidad máxima del árbol (sin limitar o de $2$ a $10$).

* `min_samples_leaf`: el número mínimo de ejemplos por hoja. Probaremos a partir de $5$ para evitar sobreajuste.

* `ccp_alpha`: parámetro para controlar la pospoda del árbol.

In [ ]:
decision_tree_classifier = DecisionTreeClassifier(random_state=random_state)

criterion = ["gini", "entropy"]
max_depth = [None, 2, 3, 4, 5, 6, 7, 8, 9, 10]
min_samples_leaf = [5, 6, 7, 8, 9]
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]

decision_tree_clf = utils.optimize_params(decision_tree_classifier,
                                          train_X, train_y, cv,
                                          scoring="recall",
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          ccp_alpha=ccp_alpha)



Podemos observar que existen diversas configuraciones de parámetros que nos dan la misma puntuación máxima. En este caso tomaremos la opción de no limitar la profundidad del árbol excepto por obligar a que las hojas tengan al menos $5$ ejemplos para evitar sobreajustar totalmente.

## 3.3. Adaptative Boosting (AdaBoost)

Para AdaBoost, deberemos decidir los parámetros del propio algoritmo así como los del clasificador usado como base, que será un árbol de decisión de poca profundidad. Estos serán:

* `learning_rate`: la tasa de aprendizaje.

* `n_estimators`: probaremos con $50$ y $75$ clasificadores.

* Y para el árbol de decisión:
    * `criterion`: si usar `gini` o `entropia` para tomar la decisión de particionar.

    * `max_depth`: la profundidad máxima del árbol, que deberá ser pequeña.

    * `ccp_alpha`: el parámetro de penalización para la pospoda.

In [ ]:
base_estimator = DecisionTreeClassifier(random_state=random_state)

adaboost_classifier = AdaBoostClassifier(random_state=random_state, base_estimator=base_estimator)

learning_rate = [0.95, 1.0]
n_estimators = [50, 75]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

adaboost_clf = utils.optimize_params(adaboost_classifier,
                                     train_X, train_y, cv,
                                     scoring="recall",
                                     learning_rate=learning_rate,
                                     n_estimators=n_estimators,
                                     base_estimator__criterion=criterion,
                                     base_estimator__max_depth=max_depth,
                                     base_estimator__ccp_alpha=ccp_alpha)

Podemos observar que los mejores resultados se dan cuando se aplica pospoda, usando como criterio `entropy`. También observamos que el aumento del número de clasificadores de $50$ a $75$ no tiene demasiado impacto, por lo que entre estas dos opciones elegiremos la de menor complejidad.



## 3.4. Bootstrap Aggregating (Bagging)

Para el caso del *bagging* aprovecharemos los hiperparámetros por defecto (queremos muestreo con reemplazo y la totalidad de las características), y solo modificaremos el número de clasificadores (`n_estimators`) y el criterio del árbol de decisión usado como base (`criterion`), que esta vez interesa que sea profundo por lo que mantendremos los parámetros relacionados con la prepoda (`max_depth`, `min_samples_leaf`, etc) o pospoda (`ccp_alpha`) en su valor por defecto.


In [ ]:
base_estimator = DecisionTreeClassifier(random_state=random_state)

bagging_classifier = BaggingClassifier(random_state=random_state, base_estimator=base_estimator)

n_estimators = [100, 250, 500]
criterion = ["gini", "entropy"]

bagging_clf = utils.optimize_params(bagging_classifier,
                                    train_X, train_y, cv,
                                    scoring="recall",
                                    n_estimators=n_estimators,
                                    base_estimator__criterion=criterion)

En este caso parece claro que cuantos más clasificadores se usen, mejor resultado se alcanza, aunque la mejora entre emplear $250$ o $500$ es casi inapreciable por lo que puede no merecer la pena. Una vez más, usando entropía conseguimos mejores resultados. También es interesante observar que se ha conseguido un $100\%$ clasificando los datos de entrenamiento, lo cual indica un gran sobreajuste a estos.

## 3.5. Random Forests

En este algoritmo al igual que en el resto basados en árboles probaremos cuál de los criterios para medir la calidad de una partición es mejor. Además, al tomar para cada árbol un subconjunto aleatorio de características, deberemos elegir cuántas tomar. El muestreo nuevamente será con reemplazo. Por lo tanto los hiperparámetros que variaremos serán:

* `criterion`: `gini` o `entropy`.

* `max_features`: $\sqrt{\mathit{n\_features}}$ o $log_2(\mathit{n\_features})$.

* `n_estimators`: el número de clasificadores base.

In [ ]:
random_forest_classifier = RandomForestClassifier(random_state=random_state)

n_estimators = [100, 150]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf = utils.optimize_params(random_forest_classifier,
                                          train_X, train_y, cv,
                                          scoring="recall",
                                          n_estimators=n_estimators,
                                          criterion=criterion,
                                          max_features=max_features)

Los resultados indican que a mayor número de árboles mejor resultado, como era de esperar. También podemos observar que esta vez el criterio `gini` se impone a la entropía (`entropy`). En cuanto al número de características seleccionadas, debido al bajo número de características totales con los que contamos en este conjunto de datos, ambos métodos son en esencia idénticos y la ligera ventaja que encontramos que tiene la raíz cuadrada es seguramente debido al azar. Por último, observamos que se ha sobreajustado a los datos de entrenamiento, consiguiendo un $100\%$ para estos.

## 3.6. Gradient Tree Boosting (Gradient Boosting)

Para este algoritmo, tendremos en cuenta al igual que con el resto de *ensembles* el número de clasificadores y la tasa de aprendizaje. Para los árboles usados como base intentaremos ajustar la profundidad máxima y el parámetro de complejidad para la poda. Al estar hablando ahora de árboles de regresión en lugar de clasificación usaremos como criterio el error cuadrado medio con y sin la mejora de Friedman. Los parámetros del algoritmo que variaremos, por tanto, son:

* `criterion`
* `n_estimators`
* `learning_rate`
* `max_depth`
* `ccp_alpha`

In [ ]:
gradient_boosting_classifier = GradientBoostingClassifier(random_state=random_state)

learning_rate = [0.01, 0.05, 0.1]
n_estimators = [100, 200]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf = utils.optimize_params(gradient_boosting_classifier,
                                          train_X, train_y, cv,
                                          scoring="recall",
                                          learning_rate=learning_rate,
                                          n_estimators=n_estimators,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          ccp_alpha=ccp_alpha)

Si nos fijamos en los mejores modelos, encontramos que la tasa de aprendizaje preferible de entre las probadas es $0.05$. Además un mayor número de árboles mejora los resultados como viene siendo habitual. Ambos criterios de calidad de las particiones probados no presentan una diferencia significativa en los resultados. Los árboles que mejor han funcionado han sido los de profundidad $3$ sin pospoda (`ccp_alpha` $=0$).

## 3.7. Histogram-Based Gradient Boosting (Histogram Gradient Boosting)



Este último algoritmo se diferencia del anterior en que se reduce el número de umbrales que se prueban para hacer las particiones mediante el uso de histogramas, lo que lo hace bastante más rápido. En esta ocasión estudiaremos los parámetros:

* `learning_rate`: la tasa de aprendizaje. En esta ocasión es viable usar valores más bajos debido al menor tiempo de entrenamiento con respecto a *Gradient Boosting*.

* `max_leaf_nodes`: esta vez usaremos el máximo número de nodos hoja: otra forma de controlar el tamaño de los árboles en lugar de por su profundidad máxima.

In [ ]:
histogram_gradient_boosting_classifier = HistGradientBoostingClassifier(random_state=random_state)

learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]

hist_gradient_boosting_clf = utils.optimize_params(histogram_gradient_boosting_classifier,
                                                   train_X, train_y, cv,
                                                   scoring="recall",
                                                   learning_rate=learning_rate,
                                                   max_leaf_nodes=max_leaf_nodes)

Como podemos observar, los mejores resultados se han obtenido con distinto número máximo de hojas de los árboles, por lo que podemos deducir que este hiperparámetro no es el más importante. Se usará por tanto el modelo de menor complejidad. Sin embargo, la tasa de aprendizaje que ha dado mejores resultados ha sido $0.03$.

# 4. Construcción y validación del modelo final

Por último, compararemos los resultados de cada modelo con los hiperparámetros aprendidos y entrenados con la totalidad del conjunto de entrenamiento, contra el conjunto de test que habíamos reservado al principio.

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf
}

In [ ]:
utils.evaluate_estimators(estimators, test_X, test_y)

# Conclusiones

Como podemos observar, en términos de *recall*, los modelos que proporcionan mejores resultados para este problema son el árbol de decisión y el AdaBoost; y el peor el Vecinos más cercanos. Si nos fijamos en otras métricas como el tiempo necesario para entrenamiento e inferencia, el árbol de decisión vuelve a destacar.

Ante estos resultados, debemos priorizar el **árbol de decisión** al conseguir un buen resultado y ser más simple.

---

# Mushroom Classification

Vamos a estudiar el siguiente kernel [Mushroom Classification & why it's easy to 100%ac](https://www.kaggle.com/arevel/mushroom-classification-why-it-s-easy-to-100-ac) realizando las modificaciones que consideramos necesarias:
* Traducción de la libreta.
* Uso de un script de utilidades con el código necesario para plotear gráficas y generar los modelos.
* Arreglo de **Data Leaks** encontrados.

# 1. Preliminares

Lo primero de todo será cargar las librerías para que estén disponibles posteriormente:

In [ ]:
import numpy as np 
import pandas as pd 
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import random

#Preprocesamiento de datos

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Creación de modelos y búsqueda de hiperparámetros

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline, Pipeline

#Validación y visualización de métricas

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, precision_score, recall_score, auc
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score

import utilidad_grupor as utils

**Librerías usadas** 

* [Numpy](https://numpy.org/): Estudiar y trabajar con los datos
* [Pandas](https://pandas.pydata.org/): Trabajr con el conjunto de datos
* [Sklearn](https://scikit-learn.org/stable/): Crear y evaluar los modelos
* [Seaborn](https://seaborn.pydata.org/): Visualizar gráficas con los datos
* [Matplotlib](https://matplotlib.org/): Visualizar gráficas con los datos


# 2. Carga de datos

En esta libreta trabajaremos con distintos tipos de modelos que evaluaremos posteriormente para tratar de clasificar las setas en dos clases:
* Comestibles (edible) clasificadas como `e`.
* Venenosas (poisonous) clasificadas como `p`.

Fijamos también una semilla para que los experimentos sean reproducibles

In [ ]:
seed = 27912

Cargamos el conjunto de datos y visualizamos sus elementos

In [ ]:
df = pd.read_csv('../input/mushroom-classification/mushrooms.csv')
dataset = df.values
df.sample(5, random_state=seed)

Mostramos el tamaño del conjunto de datos

In [ ]:
df.shape

Comprobamos si hay valores nulos

In [ ]:
df.isnull().sum()

Nuestra variable clase, como vemos, se llama `class`. Lo que haremos será separar esta variable de las variables predictoras

In [ ]:
names = list(df.columns)
x = df[names[1:]]
y = df['class']

En la libreta en la cual nos hemos basado, se pasaba directamente a realizar un análisis exploratorio de los datos teniendo en cuenta la totalidad del conjunto de datos.

Esto significa que exploraba con el mismo conjunto de datos con los que luego iba a validar el modelo, causando así un **Data Leak**. Para solucionar este problema, hemos decidido dividir primero nuestro conjunto de datos en dos subconjuntos de entrenamiento y test.

De esta manera, trabajaremos en el análisis exploratorio con este conjunto de entrenamiento, y el de test permanecerá sin usarse hasta el apartado de selección de modelos.

Por tanto, antes de comenzar el análisis exploratorio de los datos, dividiremos nuestro conjunto de datos en otros dos subconjuntos, uno de entrenamiento y otro de prueba, con los siguientes porcentajes:

* Conjunto de entrenamiento: **80%**
* Conjunto de prueba: **20%**

Mediante este proceso, nos aseguraremos de que los resultados posteriores del proceso de validación han sido obtenidos de una manera correcta.

In [ ]:
#añadimos la seed al random state para evitar sesgo y que los experimentos sean reproducibles

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=seed)

Seguidamente, lo que haremos será unir los conjuntos `X_train` e `y_train` para obtener el conjunto de datos de entrenamiento. Haremos los mismo para `X_test` e `y_test`, juntando así el conjunto de datos de test.

In [ ]:
train = utils.join_dataset(x_train, y_train)
test = utils.join_dataset(x_test, y_test)

# 3. Análisis exploratorio de los datos

El análisis exploratorio de datos es un paso fundamental a la hora de comprender los datos con los que vamos a trabajar.

El objetivo de este análisis es explorar, describir y visualizar la naturaleza de los datos recogidos mediante la aplicación de técnicas simples de resumen de datos y métodos gráficos, para observar las posibles relaciones entre las variables de nuestro conjunto de datos.

Comenzemos mostrando la distribución de la clase

In [ ]:
# Arreglados Data Leaks

colors = ('#EF8787','#9CF29C')
palette = sns.set_palette(sns.color_palette(colors))

utils.plot_class_distribution(train, 'class', ["Venenoso", "Comestible"], colors, 'Mushroom Class Distribution')

Como podemos observar, el problema está cerca de ser balanceado, pero encontramos algún ejemplo más de setas venenosas que comestibles

Veamos ahora una serie de diagramas de barras que nos relacionarán la distribución de cada variable predictora con sus posibles valores junto a la variable clase, y el número de ejemplos de las mismas

In [ ]:
# Arreglados Data Leaks

utils.plot_feature_distribution(train, 'class', train.columns, palette)

Podemos apreciar así, como distintas variables predictoras nos darán mas información que otras. Podemos ver también que solo hay un tipo de **`veil-type`** en esa variable, por lo que en el posterior **preprocesamiento de datos** que llevemos a cabo, eliminaremos esa variable

Como las variables de nuestro conjunto de datos son categóricas, llevaremos a cabo un mapa de calor para observar la relación entre estas variables.

Antes, deberemos crear un `LabelEncoder` que transforme nuestras variables con un valor entre $0$ y el $ nclases - 1 $.

In [ ]:
# Arreglados Data Leaks

labelencoder=LabelEncoder()
train_enc = train.copy()
for column in train.columns:
    train_enc[column] = labelencoder.fit_transform(train[column])

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(train_enc.corr(),linewidths=.1,annot=True, cmap="magma")

De esta manera, podremos ver las variables que se encuentran más correladas unas con otras. A simple vista, encontramos diversas parejas de variables que están altamente relacionadas unas con otras.

* `veil-color y gill-attachment` con una correlación del 89%
* `ring-type y bruises` con una correlación del 69%
* `ring-type y gill-color` con una correlación del 63%
* `spore-print-color y gill-size` con una correlación del 62%

De este estudio extraemos que las variables `bruises y gill-color` dependen ambas de `ring-type`, y consideramos que podrían ser eliminadas. Pese a que en el procesamiento no se haya llevado a cabo dicha acción, la implementaremos.

Como hemos mencionado anteriormente, la variable `veil-type` será eliminada puesto que no aporta ninguna información relevante.

# 4. Preprocesamiento de los datos

En esta etapa limpiaremos y organizaremos los datos de manera adecuada para entrenar a nuestro modelo basándonos en las observación que hemos realizado en el análisis exploratorio de datos previo. Por ello, en este conjunto de datos nos centraremos en la selección de variables adecuadas para conseguir reducir el número de estas.

Para realizar este proceso, haremos uso de un **Pipeline**. Este Pipeline será el encargado de aplicar las transformaciones que hemos decidido a nuestro conjunto de datos.

## 4.1. Eliminacion de variables

Como hemos comentado anteriormente, será recomendable eliminar la variable que no aporta información `veil-type` y las dos variables que van relacionadas con `ring-type`, como son `bruises y gill-color`.

In [ ]:
#Añadimos pipeline para preprocesamiento 

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

preproc = ColumnTransformer([("", "drop", ["bruises","gill-color","veil-type"])], remainder="passthrough")

In [ ]:
preproc.fit_transform(x_train)
preproc.fit_transform(x_test)

## 4.2. Codificación de los datos

Como nuestro conjunto de datos está formado por variables categóricas, para trabajar con ellos debemos codificarlos. A la hora de realizar este proceso, debemos tener en cuenta que para algunos modelos, codificar los valores directamente como números pueden crear sesgo hacia las variables con mayor valor.

Para evitar eso, haremos uso de `OneHotEncoder`

In [ ]:
#Train
ohe_x_train = OneHotEncoder(drop='first').fit(x_train)
ohe_x_train = ohe_x_train.transform(x_train).toarray()

aux = y_train.values.reshape(-1, 1)
ohe_y_train = OneHotEncoder(drop='first').fit(aux)
ohe_y_train = ohe_y_train.transform(aux).toarray()
ohe_y_train = ohe_y_train.flatten()

#Test
ohe_x = OneHotEncoder(drop='first').fit(x_test)
ohe_x = ohe_x.transform(x_test).toarray()

aux = y_test.values.reshape(-1, 1)
ohe_y = OneHotEncoder(drop='first').fit(aux)
ohe_y = ohe_y.transform(aux).toarray()
ohe_y = ohe_y.flatten()

No es necesario normalizar los datos en este caso, puesto que nuestras variables son categóricas

Pasemos ahora a la selección de modelos, donde emplearemos para cada uno el **Pipeline** definido.

# 5. Selección de modelos

Hay muchos modelos diferentes y muchas variaciones diferentes de cada modelo, para elegir los que tendrán un mejor rendimiento, debemos considerar cuál se ajustará mejor a nuestro conjunto de datos.

Nuestro conjunto de datos está equilibrado, la entrada y la salida son categóricas y tenemos alrededor de 8.000 instancias. Teniendo esto en cuenta usaremos los siguientes modelos:

* Regresión Logística
* Naive Bayes
* Random Forest
* KNN

En primer lugar, definimos una función para mostrar los resultados obtenido por cada uno de los modelos.

In [ ]:
models = ['LogisticRegression','NaiveBayes','RandomForest','KNearestNeighbors']

scores = [None] * len(models)

## 5.1. Regresión Logística

In [ ]:
from sklearn.model_selection import cross_val_score

lr = LogisticRegression()
lr.fit(ohe_x_train, ohe_y_train)
y_pred = lr.predict(ohe_x)
accuracy = lr.score(ohe_x, ohe_y)

utils.show_results(models, scores, ohe_x, ohe_y, lr, y_pred,"LogisticRegression")

Hemos obtenido un 100% de precisión, por lo que podríamos estar teniendo un problema de sobreajuste. Para evaluar mejor los modelos de ahora en adelante, usaremos validación cruzada. Al hacerlo, es probable que obtengamos peores resultados, pero estos serán más fiables.

In [ ]:
score_list = cross_val_score(lr,ohe_x,ohe_y, cv=10)
score = np.mean(score_list)
print (score)

# we swap the score obtained before with the cross_val_score
scores[0] = score

## 5.2. Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(ohe_x_train, ohe_y_train)
preds= nb.predict(ohe_x)
utils.show_results(models, scores, ohe_x, ohe_y, nb, preds,"NaiveBayes")

In [ ]:
score_list = cross_val_score(nb,ohe_x,ohe_y, cv=5)
print(score)
score = np.mean(score_list)
# we swap the score obtained before with the cross_val_score
scores[1] = score

Nuevamente podemos ver cómo después de usar la validación cruzada, nuestra precisión ha disminuido esta vez hasta casi un 85%.

In [ ]:
print(score_list)

Podemos observar cómo la precisión del 99% obtenida previamente no era fiable.

### Búsqueda Grid
Ahora separamos los datos en carpetas para la búsqueda **Grid**

In [ ]:
cv_split = TimeSeriesSplit(n_splits=5)

## 5.3. Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1)
rf_params = {
    'model__n_estimators': list(range(25,251,25)),
    'model__max_features': list(np.arange(0.1,0.36,0.05))
}
rf_pipe = Pipeline([
    ('scale', StandardScaler()),
    ('model', rf)
])
gridsearch_rf = GridSearchCV(estimator=rf_pipe,
                          param_grid = rf_params,
                          cv = cv_split,
                         )
gridsearch_rf.fit(ohe_x_train, ohe_y_train)

In [ ]:
rf_best_model = gridsearch_rf.best_estimator_
preds = rf_best_model.predict(ohe_x)
utils.show_results(models, scores, ohe_x, ohe_y, rf_best_model, preds,'RandomForest')

## 5.4. KNN

In [ ]:
knn = KNeighborsClassifier()
knn_params = {
    'n_neighbors': list(range(4,10)),
    'weights': ['uniform','distance']
}

gridsearch_knn = GridSearchCV(knn,
                          param_grid = knn_params,
                          cv = cv_split,
                         )
gridsearch_knn.fit(ohe_x_train, ohe_y_train)

In [ ]:
knn_best_model = gridsearch_knn.best_estimator_
preds = knn_best_model.predict(ohe_x)
utils.show_results(models, scores, ohe_x, ohe_y, knn_best_model, preds,'KNearestNeighbors')

Después de esta búsqueda **grid** podemos pensar que independientemente de la búsqueda de hiperparámetros y el ajuste de las variables el modelo es propenso a obtener precisiones muy altas.

Veamos, para terminar, un resumen de la precisión obtenida por cada uno de los modelos que hemos estudiado para resolver nuestro problema.

In [ ]:
utils.plot_accuracy(models, scores)

Como podemos apreciar, RandomForest y KNN son los que nos proporcionan una mayor precisión cuando los enfrentamos contra el conjunto de test que reservamos previamente.

# 6. ¿Precisión del 100%?

Hemos visto cómo todos los modelos probados pueden obtener altas precisiones y lo poco que varían las precisiones entre los diferentes parámetros (ilustrado con el ejemplo KNN).

Para comprender finalmente lo fácil que podemos lograr altas precisiones, trazaremos varias curvas ROC para determinados subgrupos de características aleatorias.

In [ ]:
palette = sns.set_palette(sns.color_palette('Set1')) #just to define de plot palette

for i in range(0,5):
    random.seed(i)
    randlist = list(names[x] for x in random.sample(range(0,21),k=5))
    rand_df = train[randlist]
    rand_df = pd.get_dummies(rand_df)

    utils.plot_roc(rand_df, ohe_y_train)
    
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

plt.gcf().set_size_inches(15,10)

# Conclusiones

Tanto KNN como RandomForests nos darán una precisión casi perfecta. Incluso si las características del conjunto de datos permiten tener fácilmente altas precisiones.

Siempre es importante procesar correctamente los datos y ajustar los modelos correctamente, entendiendo lo que está haciendo el programa en lugar de solo enfocarse en obtener mejores métricas.

Al hacer esto, sabremos si nuestros scores son correctos o si estamos haciendo algo mal.